# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
from pprint import pprint 

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Output File (CSV)

csvpath = os.path.join("Output_data", "output_data_cities.csv")

city_df = pd.read_csv(csvpath)

city_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
locations=city_df[["Lat","Lng"]].astype(float)
humidity = city_df["Humidity"].astype(float)


fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=80,
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig




Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

ideal_weather = city_df.loc[(city_df['Cloudiness']<=0) 
                            & (city_df['Wind Speed']<=10) & (city_df['Max Temp']<=80) & (city_df['Max Temp']>=70)]

ideal_weather_clean = ideal_weather.dropna()    ### Objective Orin. Programing
#len(city_df) = 548
len(ideal_weather)
#city_df.head()
ideal_weather_clean

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_list = []
lat_lng_hotels = ideal_weather_clean[['Lat', 'Lng']]



base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for idx, row in ideal_weather_clean.iterrows():
    
    params = {
        "location": str(row["Lat"])+","+str(row["Lng"]),
#         "rankby": "distance",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }

    
    # assemble url and make API request
    print(f"Retrieving Results for Index {idx}: {row['City_ID']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
   
    hotel_list.append(response['results'][0]['name'])
    
        
#     try:
#         print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
#         types_df.loc[index, 'name'] = results[0]['name']
#         types_df.loc[index, 'address'] = results[0]['vicinity']
#         types_df.loc[index, 'price_level'] = results[0]['price_level']
#         types_df.loc[index, 'rating'] = results[0]['rating']
        
#     except (KeyError, IndexError):
#         print("Missing field/result... skipping.")
        
#     print("------------")




Retrieving Results for Index 70: 70.
Retrieving Results for Index 88: 88.
Retrieving Results for Index 138: 138.
Retrieving Results for Index 139: 139.
Retrieving Results for Index 176: 176.
Retrieving Results for Index 297: 297.
Retrieving Results for Index 444: 444.


In [7]:
type(row['Lat'])

float

In [8]:
hotel_list

['Hotel Escarpas do Lago',
 'Hotel Castelo',
 'La Case Bambou',
 'Dandeli Lake County',
 'Hôtel La Giralda',
 'MOGOK HILL HOTEL',
 'تاکسی سرویس آرامش']

In [9]:
pprint(response)


{'html_attributions': [],
 'next_page_token': 'ATtYBwKeZG2GC_WAeIbQ42humfmHVwEhdKnXcqdzTtQzVGUKTFaM9HBlzyA68GvojH-LtQAnzZf8hBtnl54B96xIUZrB7kr0RZ63oDNC8XZrHV1e2dMnZmZvPtXqUQyt2BcMh4EPwz13Lh8Pbg7ECeL9mYkEMPGlFqKZ0g5ulFjGzVAH7EPzRDhWEUDj84vU-v0m0TypDHvmPMMB9ASSinxH1MRMiMKIT2V9k3qXXA9M3bOQFrOaoG613dOuFSgroU2ULTs5tJkCXoQzv0PIzwt6wbWgcfjwf0rTAPjZ_y_uj4glFvByT1ATZRuYSxgSkT0EvcwEzJta-OhuoOBw_g2ov4fsAKMo-XFSdyYaUMhdkH3ve-7WuI0Re2_RqnE128-zTt4Pp1OfhJffT7iePXLJ2jgvIhIeBPvPmazP_YrcA1-qPzh9fum8tbfw',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 32.8625177, 'lng': 59.2037759},
                           'viewport': {'northeast': {'lat': 32.86386753029149,
                                                      'lng': 59.20508738029152},
                                        'southwest': {'lat': 32.86116956970849,
                                                      'lng': 59.2023894197085}}},
              'icon': 'https://maps.gstatic.com/mapfiles

In [10]:
ideal_weather_clean["Hotel Name"] = hotel_list

hotel_df = ideal_weather_clean

hotel_df

C:\Users\natha\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,La Case Bambou
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Lake County
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel La Giralda
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,MOGOK HILL HOTEL
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,تاکسی سرویس آرامش


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
marker_locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map


markers = gmaps.marker_layer(marker_locations, hotel_info)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))